## Import de Bibliotecas

In [27]:
import pandas as pd
import numpy as np
from config import *
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import statistics as stat

pd.set_option('display.max_columns', 500)

## Leitura dos dados

In [28]:
df = pd.read_csv(path_arquivo_treino)
df = df.drop(df[df.vh_weight < 50].index)
df.shape

(217636, 26)

## Análises

In [29]:
print('Probabilidade de claim > 0 na amostra é',len(df[df['claim_amount']>0])/len(df))

Probabilidade de claim > 0 na amostra é 0.10373283831719017


In [30]:
df['claim_amount'].describe()

count    217636.000000
mean        116.387763
std         736.649932
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max      141828.260000
Name: claim_amount, dtype: float64

In [31]:
df['pol_no_claims_discount'].mean()

0.04485813008877452

In [32]:
df_scatter = df[df['year']==4].dropna(subset = ['drv_age2'])

fig = px.scatter(df_scatter, x="drv_age1", y="drv_age2")
fig.show()

#### Grupo de pais

In [33]:
df[df['drv_age2']<=30]['drv_age1'].mean() 

52.85801417167459

In [34]:
df_pais = df[(df['drv_age2']<=30) & (df['drv_age1']>50) & (df['drv_age1']<70)]
df_pais['claim_amount'].describe()

count    6776.000000
mean      130.523747
std       543.435435
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max      8220.780000
Name: claim_amount, dtype: float64

In [35]:
print('A média de risco no grupo de pai é',df_pais['pol_no_claims_discount'].mean())

A média de risco no grupo de pai é 0.017887544273907922


In [36]:
print('Estando no grupo de pais, a probabilidade de claim >0 é',len(df_pais[df_pais['claim_amount']>0])/len(df_pais))

Estando no grupo de pais, a probabilidade de claim >0 é 0.11998229043683589


In [37]:
print('Porcentagem do claim no grupo de pais é',(df_pais['claim_amount'].sum()/df['claim_amount'].sum())*100,'%')

Porcentagem do claim no grupo de pais é 3.491603143424666 %


#### Grupo conjuges

In [38]:
df_conjuges = df[(df['drv_age1']>65) & (df['drv_age2']>45) & (df['drv_age2']<75)] 

In [39]:
df_conjuges['claim_amount'].describe()

count    9847.000000
mean      121.972489
std       489.555172
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max      7770.160000
Name: claim_amount, dtype: float64

In [40]:
print('A média de risco no grupo de conjuges é',df_conjuges['pol_no_claims_discount'].mean())

A média de risco no grupo de conjuges é 0.015559459733929084


In [41]:
print('Estando no grupo de conjuges, a probabilidade de claim >0 é',len(df_conjuges[df_conjuges['claim_amount']>0])/len(df_conjuges))

Estando no grupo de conjuges, a probabilidade de claim >0 é 0.11181070376764497


In [42]:
print('Porcentagem do claim no grupo de conjuges é',(df_conjuges['claim_amount'].sum()/df['claim_amount'].sum())*100,'%')

Porcentagem do claim no grupo de conjuges é 4.741631179165518 %


#### Grupo de uso privado com cobertura mínima

In [43]:
df_work_min = df[(df['pol_usage']=='WorkPrivate')&(df['pol_coverage']=='Min')]
df_work_min['claim_amount'].describe()

count    12424.000000
mean        25.016351
std        208.628648
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max       7643.530000
Name: claim_amount, dtype: float64

In [44]:
print('Estando no grupo de uso privado com cobertura mínima a probabilidade de claim >0 é',len(df_work_min[df_work_min['claim_amount']>0])/len(df_work_min))

Estando no grupo de uso privado com cobertura mínima a probabilidade de claim >0 é 0.022295556986477786


In [45]:
print('A média de risco no grupo de uso privado com cobertura mínima é',df_work_min['pol_no_claims_discount'].mean())

A média de risco no grupo de uso privado com cobertura mínima é 0.06830932066967205


In [46]:
print('Porcentagem do claim no grupo de uso privado com cobertura mínima é',(df_work_min['claim_amount'].sum()/df['claim_amount'].sum())*100,'%')

Porcentagem do claim no grupo de uso privado com cobertura mínima é 1.2270078559623931 %


#### Grupo de uso profissional e máxima cobertura de seguro

In [47]:
df_prof_max = df[(df['pol_usage']=='Professional')&(df['pol_coverage']=='Max')]
df_prof_max['claim_amount'].describe()

count    10280.000000
mean       208.157601
std        766.992348
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max      27256.430000
Name: claim_amount, dtype: float64

In [48]:
print('A média de risco no grupo de  uso profissional com máxima cobertura é',df_prof_max['pol_no_claims_discount'].mean())

A média de risco no grupo de  uso profissional com máxima cobertura é 0.04138871595330719


In [49]:
print('Estando no grupo de uso profissional com máxima cobertura, a probabilidade de claim >0 é',len(df_prof_max[df_prof_max['claim_amount']>0])/len(df_prof_max))

Estando no grupo de uso profissional com máxima cobertura, a probabilidade de claim >0 é 0.163715953307393


In [50]:
print('Porcentagem do claim no grupo de  uso profissional com máxima cobertura é',(df_prof_max['claim_amount'].sum()/df['claim_amount'].sum())*100,'%')

Porcentagem do claim no grupo de  uso profissional com máxima cobertura é 8.447872188295094 %


In [51]:
def f(x):
    d = {}
    d['claim_pct'] = x['claim_amount'].sum()/df['claim_amount'].sum()
    d['prob_claim'] = x[x['claim_amount']>0]['id_policy'].count()/x['id_policy'].count()
    d['count'] = x['id_policy'].count()
    return pd.Series(d, index=['claim_pct', 'prob_claim','count'])


df.groupby(['vh_type','pol_coverage']).apply(f)

claim_pct  prob_claim     count
vh_type    pol_coverage                                 
Commercial Max            0.033953    0.135526    6080.0
           Med1           0.006183    0.073415    3092.0
           Med2           0.009188    0.077682    4660.0
           Min            0.003536    0.019048    3780.0
Tourism    Max            0.818209    0.124770  136796.0
           Med1           0.032351    0.066774   16788.0
           Med2           0.082705    0.077775   33224.0
           Min            0.013876    0.024062   13216.0

In [54]:
df.groupby('pol_usage')['vh_weight'].mean()

pol_usage
AllTrips        1149.200000
Professional    1183.559140
Retired         1138.569368
WorkPrivate     1155.624148
Name: vh_weight, dtype: float64

In [52]:
df.groupby(['pol_usage','pol_coverage']).apply(f)

claim_pct  prob_claim    count
pol_usage    pol_coverage                                
AllTrips     Max            0.002633    0.226744    172.0
             Med1           0.000021    0.250000      8.0
             Med2           0.000000    0.000000     16.0
             Min            0.000000    0.000000      4.0
Professional Max            0.084479    0.163716  10280.0
             Med1           0.002087    0.075000   1040.0
             Med2           0.005104    0.093750   1952.0
             Min            0.001369    0.037688    796.0
Retired      Max            0.231611    0.115339  41816.0
             Med1           0.006386    0.049011   4448.0
             Med2           0.023828    0.061505   9300.0
             Min            0.003773    0.022004   3772.0
WorkPrivate  Max            0.533438    0.125232  90608.0
             Med1           0.030039    0.072998  14384.0
             Med2           0.062961    0.082319  26616.0
             Min            0.012270    0.022296  12424.0